In [ ]:
import numpy as np 
import pandas as pd 
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Dense , Conv2D , GlobalAveragePooling2D , Dropout , BatchNormalization , Input 
from tensorflow.keras.models import Model 

## read the data 

In [ ]:
data_dir = '../input/tsp-cv'
train_df = pd.read_csv(os.path.join(data_dir , 'train.csv'))
test_df = pd.read_csv(os.path.join(data_dir , 'test.csv'))

In [ ]:
train_df.info()

## get image paths  
each row in the dataframe has file name feature so we concat it with the data dir path to git the images paths 

In [ ]:
train_df['image_path'] = data_dir + '/'+ train_df['filename'] 
test_df['image_path'] = data_dir + '/'+ test_df['filename']

In [ ]:
fig = plt.figure(figsize=(10, 7))
for i in range(10) : 
    fig.add_subplot(2, 2, 1)
    img = mpimg.imread(train_df['image_path'][i])
    imgplot = plt.imshow(img)
    plt.show()


In [ ]:
train_data , val_data  = train_test_split(train_df , test_size = .2 , random_state = 123 , shuffle = True )

## data generators 

In [ ]:
train_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1/255  , preprocessing_function = tf.keras.applications.inception_v3.preprocess_input)
val_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1/255 , preprocessing_function = tf.keras.applications.inception_v3.preprocess_input)
test_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale= 1/255 ,preprocessing_function = tf.keras.applications.inception_v3.preprocess_input)


In [ ]:
X_train = train_generator.flow_from_dataframe(train_data , x_col = 'image_path' , y_col ='distance' , 
                                              class_mode ='raw' , batch_size = 64 , target_size  = (224,224) ) 
X_val = val_generator.flow_from_dataframe(val_data , x_col = 'image_path' , y_col ='distance' , 
                                              class_mode = 'raw' , batch_size = 64 , target_size  = (224,224) )
X_test = test_generator.flow_from_dataframe(test_df , x_col = 'image_path' ,
                                              class_mode = None , batch_size = 64 , target_size  = (224,224) )

## load pretrained model 

In [ ]:
pretrain_model = tf.keras.applications.InceptionV3(include_top = False , weights = 'imagenet' , input_shape = (224,224,3) ) 

In [ ]:
# we freeze all model weights except the last two conv layers 
for layer in pretrain_model.layers[:-17]:
    layer.trainable = False


## model 

In [ ]:
input_layer = Input(shape = (224 ,224 ,3))
X = pretrain_model(input_layer) 
X = GlobalAveragePooling2D()(X)
X = Dense(1024 , activation = 'relu')(X)
X = Dropout(.5)(X)
X = Dense(64 ,activation ='relu')(X)
X = Dense(64 ,activation ='relu')(X)
X = Dropout(.2)(X)
out = Dense(1)(X) 
model = Model(input_layer , out)

In [ ]:
model.summary()

## custom loss function 

In [ ]:
def rmse(y_true, y_pred):
    y_true, y_pred = tf.cast(y_true , tf.float32) , tf.cast(y_pred , tf.float32)
    squared_difference = tf.square(y_true - y_pred)
    return tf.sqrt(tf.reduce_mean(squared_difference, axis=-1))  # Note the `axis=-1`

In [ ]:
model.compile(optimizer = 'adam' , loss = rmse, metrics = ['mse'])


In [ ]:
early_stopping = tf.keras.callbacks.EarlyStopping(patience = 5 , monitor = 'val_loss')
reduce_lr_in_plateau = tf.keras.callbacks.ReduceLROnPlateau( monitor="val_loss",patience=5)

In [ ]:
history = model.fit(X_train , epochs = 50 , validation_data = X_val , batch_size = 64 , callbacks = [early_stopping ,reduce_lr_in_plateau ])